# RAG-Vision: Image → Caption/Summary → Embeddings ETL Pipeline

## Overview

This notebook implements a **multimodal RAG pipeline** that extracts meaning from images using Vision-Language Models (VLMs). Unlike text-only RAG pipelines, RAG-Vision enables semantic search over visual content like charts, diagrams, infographics, and screenshots.

### Pipeline Flow

```
Reader2Image → VLM Captioning → Splitter → Sentence Embeddings → Database
```

### Key Use Cases

- **📊 Slide Decks with Charts**: Extract meaning from presentation visuals and diagrams
- **📋 Scanned Forms**: Process documents with visual elements and handwritten content
- **📈 Infographics**: Analyze data visualizations and graphical information
- **🖥️ EHR/Portal Screenshots**: Extract information from healthcare system interfaces
- **📐 Technical Diagrams**: Process architectural drawings, flowcharts, and schematics
- **🎨 Product Images**: Enable visual search in e-commerce catalogs

### Innovation: Multimodal Retrieval

Traditional RAG systems are **text-blind** to visual content. RAG-Vision solves this by:

1. **VLM Captioning**: Use Qwen2-VL to generate contextual descriptions of images
2. **Dual Storage**: Keep both VLM captions and OCR text (if available)
3. **Rich Metadata**: Tag with `has_image=true`, `figure_id`, `slide_no`, dimensions
4. **Semantic Embeddings**: Convert visual descriptions into searchable vectors

### Why This Matters

Studies show that **65-70% of business documents** contain meaningful visual elements:
- Charts and graphs conveying data trends
- Diagrams explaining processes
- Screenshots showing UI/UX
- Tables with structured information

**Without RAG-Vision**, these insights are lost in retrieval systems.

---

In [1]:
# Download the Spark NLP Python wheel
!wget https://s3.us-east-1.amazonaws.com/auxdata.johnsnowlabs.com/public/tmp/sparknlp_rc/spark_nlp-6.2.0rc1-py2.py3-none-any.whl

# Download the Spark NLP assembly JAR
!wget https://s3.us-east-1.amazonaws.com/auxdata.johnsnowlabs.com/public/tmp/sparknlp_rc/spark-nlp-assembly-6.2.0-rc1.jar

--2025-10-22 21:17:57--  https://s3.us-east-1.amazonaws.com/auxdata.johnsnowlabs.com/public/tmp/sparknlp_rc/spark_nlp-6.2.0rc1-py2.py3-none-any.whl
Resolving s3.us-east-1.amazonaws.com (s3.us-east-1.amazonaws.com)... 52.216.208.104, 16.15.202.170, 52.216.92.149, ...
Connecting to s3.us-east-1.amazonaws.com (s3.us-east-1.amazonaws.com)|52.216.208.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 743293 (726K) [binary/octet-stream]
Saving to: ‘spark_nlp-6.2.0rc1-py2.py3-none-any.whl.8’

spark_nlp-6.2.0rc1- 100%[===================>] 725.87K  2.16MB/s    in 0.3s    

2025-10-22 21:17:58 (2.16 MB/s) - ‘spark_nlp-6.2.0rc1-py2.py3-none-any.whl.8’ saved [743293/743293]

--2025-10-22 21:17:58--  https://s3.us-east-1.amazonaws.com/auxdata.johnsnowlabs.com/public/tmp/sparknlp_rc/spark-nlp-assembly-6.2.0-rc1.jar
Resolving s3.us-east-1.amazonaws.com (s3.us-east-1.amazonaws.com)... 52.217.205.32, 52.216.208.104, 16.15.202.170, ...
Connecting to s3.us-east-1.amazonaws.com

In [2]:
!pip install spark_nlp-6.2.0rc1-py2.py3-none-any.whl

Processing ./spark_nlp-6.2.0rc1-py2.py3-none-any.whl
spark-nlp is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


Downlading Files

In [3]:
base_url = "https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/training-spark-nlp-v6-readers/tutorials/Certification_Trainings/Public/data/readers"

In [4]:
!mkdir pdf-files

mkdir: cannot create directory ‘pdf-files’: File exists


In [5]:
!wget "{base_url}/pdf-with-2images.pdf" -P pdf-files

--2025-10-22 21:26:12--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/training-spark-nlp-v6-readers/tutorials/Certification_Trainings/Public/data/readers/pdf-with-2images.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8002::154, 2606:50c0:8000::154, 2606:50c0:8003::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8002::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 130182 (127K) [application/octet-stream]
Saving to: ‘pdf-files/pdf-with-2images.pdf.1’

pdf-with-2images.pd 100%[===================>] 127.13K  --.-KB/s    in 0.03s   

2025-10-22 21:26:13 (4.26 MB/s) - ‘pdf-files/pdf-with-2images.pdf.1’ saved [130182/130182]



## 📦 Step 0: Import Dependencies

Import specialized libraries for multimodal processing:

- **PySpark**: Distributed data processing
- **Spark NLP**: Text processing components
- **Reader2Image**: Image ingestion from various sources
- **Qwen2VLTransformer**: Vision-Language Model for image understanding
- **BertSentenceEmbeddings**: Convert captions to searchable vectors

In [6]:
# Import PySpark dependencies
from pyspark.sql import SparkSession, functions as F
from pyspark.ml import Pipeline

# Import Spark NLP components
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import (
    SentenceDetector,
    BertSentenceEmbeddings
)

# Vision-Language Model for image captioning
from sparknlp.annotator import Qwen2VLTransformer

# Image reader for various formats
from sparknlp.reader.reader2image import Reader2Image

print("✅ All dependencies imported successfully")

✅ All dependencies imported successfully


## 🚀 Step 1: Initialize Spark Session

Configure Spark with enhanced settings for image processing:

**Key Configurations**:
- **Driver Memory**: 16GB (images are memory-intensive)
- **Max Result Size**: 2GB (for large image batches)
- **Serializer**: Kryo (efficient for binary data)
- **Spark NLP JAR**: Required for all annotators

In [7]:
def get_spark_session():
    """
    Create and configure a Spark session optimized for image processing.

    Returns:
        SparkSession: Configured Spark session for RAG-Vision pipeline
    """
    builder = SparkSession.builder \
        .appName("RAG-Vision: Image → Caption → Embeddings ETL") \
        .config("spark.driver.memory", "16G") \
        .config("spark.driver.maxResultSize", "2000M") \
        .config("spark.kryoserializer.buffer.max", "2000M") \
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
        .config("spark.jars",
                "../jars/spark-nlp-assembly-6.2.0-rc1.jar")  # ⚠️ Update this path!

    return builder.getOrCreate()

# Initialize Spark session
spark = get_spark_session()
print("✅ Spark session initialized successfully")

# Create empty DataFrame for Reader2Image initialization
empty_df = spark.createDataFrame([], "string").toDF("text")

25/10/22 21:26:14 WARN Utils: Your hostname, danilo-ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.100.75 instead (on interface enp131s0)
25/10/22 21:26:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/10/22 21:26:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


✅ Spark session initialized successfully


## 📸 Step 2: Image Ingestion with Reader2Image

**Reader2Image** is a specialized component that extracts images from various sources:

### Supported Formats:
- **Direct Images**: PNG, JPG, JPEG, TIFF, BMP
- **PDF Documents**: Extracts embedded images from PDFs
- **PowerPoint**: Extracts slides as images from PPTX files
- **Directories**: Batch process entire folders

### Output Annotations:
Each image becomes an `AnnotationImage` with rich metadata:
- **source/path**: Original file location
- **width/height**: Image dimensions
- **nChannels**: Color channels (1=grayscale, 3=RGB, 4=RGBA)
- **origin**: Byte array of image data

### Configuration:
- **Path-based mode**: Read from file system (shown below)
- **Column-based mode**: Read from DataFrame with binary image data

In [8]:
# Configure input path for images
# Options:
#   - Single image file: "datasets/chart.png"
#   - Image directory: "datasets/images/"
#   - PDF with images: "datasets/presentation.pdf"
#   - PowerPoint: "datasets/slides.pptx"

images_path = "./pdf-files/pdf-with-2images.pdf"

# Initialize Reader2Image
reader2image = Reader2Image() \
    .setContentType("application/pdf") \
    .setContentPath(images_path) \
    .setOutputCol("image") \
    .setUserMessage("Describe the image with 5 to 6 words.")

print(f"✅ Reader2Image configured to read from: {images_path}")
print("\n📝 Supported content types:")
print("   • 'image/png' - PNG images")
print("   • 'image/jpeg' - JPG/JPEG images")
print("   • 'application/pdf' - PDF documents")
print("   • 'application/vnd.ms-powerpoint' - PowerPoint presentations")

# Execute Reader2Image to load images
df_in = empty_df
df_images = Pipeline(stages=[reader2image]).fit(df_in).transform(df_in)

print("\n✅ Images loaded successfully")
print(f"📊 Number of images extracted: {df_images.count()}")

✅ Reader2Image configured to read from: ./pdf-files/pdf-with-2images.pdf

📝 Supported content types:
   • 'image/png' - PNG images
   • 'image/jpeg' - JPG/JPEG images
   • 'application/pdf' - PDF documents
   • 'application/vnd.ms-powerpoint' - PowerPoint presentations

✅ Images loaded successfully


[Stage 1:>                                                          (0 + 1) / 1]

📊 Number of images extracted: 2


In [9]:
df_images.show()

+--------------------+--------------------+---------+
|            fileName|               image|exception|
+--------------------+--------------------+---------+
|pdf-with-2images.pdf|[{image, pdf-with...|     NULL|
|pdf-with-2images.pdf|[{image, pdf-with...|     NULL|
+--------------------+--------------------+---------+



## 🤖 Step 3: Vision-Language Model Captioning

This is the **core innovation** of RAG-Vision. We use **Qwen2-VL**, a state-of-the-art Vision-Language Model, to generate contextual descriptions of images.

### What is Qwen2-VL?

**Qwen2-VL** is a multimodal transformer that:
- **Understands Visual Content**: Recognizes objects, text, charts, diagrams
- **Generates Captions**: Creates natural language descriptions
- **Context-Aware**: Captures semantic meaning, not just objects
- **Chart-Aware**: Can describe axes, labels, trends in data visualizations

### How It Works:
1. **Vision Encoder**: Processes image pixels into visual features
2. **Language Decoder**: Generates text from visual features
3. **Cross-Attention**: Aligns visual and textual representations

### Example Outputs:
- **Chart**: "A bar chart showing quarterly revenue growth from Q1 to Q4, with Q4 showing the highest value at $2.3M"
- **Diagram**: "A flowchart depicting the customer onboarding process with 5 steps from registration to activation"
- **Form**: "A medical intake form with sections for patient information, insurance details, and medical history"

### Configuration Options:
- **Prompt Engineering**: Guide the model with specific instructions
- **Temperature**: Control output randomness (0.0 = deterministic)
- **Max Length**: Limit caption length for consistency

In [10]:
# Configure Qwen2-VL for image captioning
vlm = Qwen2VLTransformer.pretrained() \
    .setInputCols("image") \
    .setOutputCol("vlm_caption")

print("✅ Qwen2-VL configured for image captioning")
print("\n🎯 VLM Capabilities:")
print("   • Object Recognition: Identifies objects, people, scenes")
print("   • OCR: Reads text within images")
print("   • Chart Understanding: Describes data visualizations")
print("   • Contextual Descriptions: Captures semantic meaning")
print("   • Spatial Relationships: Understands layout and positioning")

print("\n💡 Best Practices:")
print("   • Use clear prompts for specific domains (medical, technical, etc.)")
print("   • Set temperature=0.0 for consistent captions")
print("   • Keep captions retrieval-oriented (avoid creative descriptions)")

qwen2_vl_2b_instruct_int4 download started this may take some time.
Approximate size to download 1.4 GB
[ | ]

25/10/22 21:26:40 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.
25/10/22 21:26:40 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.


qwen2_vl_2b_instruct_int4 download started this may take some time.
Approximate size to download 1.4 GB
Download done! Loading the resource.
[OK!]
✅ Qwen2-VL configured for image captioning

🎯 VLM Capabilities:
   • Object Recognition: Identifies objects, people, scenes
   • OCR: Reads text within images
   • Chart Understanding: Describes data visualizations
   • Contextual Descriptions: Captures semantic meaning
   • Spatial Relationships: Understands layout and positioning

💡 Best Practices:
   • Use clear prompts for specific domains (medical, technical, etc.)
   • Set temperature=0.0 for consistent captions
   • Keep captions retrieval-oriented (avoid creative descriptions)


## ✂️ Step 4: Caption Splitting and Sentence Detection

VLM-generated captions may contain multiple sentences or concepts. We split them for granular embedding:

### Why Split Captions?
1. **Granular Retrieval**: Each sentence becomes a searchable chunk
2. **Better Matching**: Specific queries match specific caption parts
3. **Embedding Quality**: Shorter text → better embedding coherence

### SentenceDetector Features:
- Linguistic rules for accurate segmentation
- Abbreviation handling (Dr., Inc., Fig., etc.)
- Context-aware boundary detection

In [11]:
# Configure sentence detection for captions
splitter = SentenceDetector() \
    .setInputCols("vlm_caption") \
    .setOutputCol("sentences") \
    .setUseAbbreviations(True)  # Handle Fig., Dr., etc.

print("✅ Sentence detector configured for caption splitting")
print("\n📝 Example caption splitting:")
print("   Input: 'This chart shows revenue trends. Q4 had the highest sales.'")
print("   Output:")
print("      • 'This chart shows revenue trends.'")
print("      • 'Q4 had the highest sales.'")

✅ Sentence detector configured for caption splitting

📝 Example caption splitting:
   Input: 'This chart shows revenue trends. Q4 had the highest sales.'
   Output:
      • 'This chart shows revenue trends.'
      • 'Q4 had the highest sales.'


## 🎯 Step 5: Generate Sentence Embeddings from Captions

Convert VLM-generated captions into **dense vector representations** for semantic search:

### Model: BERT Sentence Embeddings
- **Model**: `sent_small_bert_L2_128`
- **Dimension**: 128 (good balance of speed/quality)
- **Advantages**: Fast inference, semantic understanding

### Why Embed Captions?
1. **Semantic Search**: Find images by meaning, not keywords
2. **Cross-Modal Retrieval**: Text queries → Image results
3. **Similarity Ranking**: Measure relevance scores

### Domain-Specific Models:
- **General**: `all-mpnet-base-v2` (highest quality)
- **Multilingual**: `labse` (50+ languages)
- **Medical**: `biobert_pubmed_base_cased`
- **Technical**: `scibert_scivocab_uncased`

In [12]:
# Configure BERT sentence embeddings
emb = BertSentenceEmbeddings.pretrained("sent_small_bert_L2_128", "en") \
    .setInputCols("sentences") \
    .setOutputCol("sentence_embeddings")

print("✅ BERT sentence embeddings configured")
print("   • Model: sent_small_bert_L2_128")
print("   • Dimension: 128")
print("   • Language: English")

print("\n💡 Alternative models for different use cases:")
print("   • 'all-mpnet-base-v2': Highest quality (384 dim)")
print("   • 'labse': Multilingual support (768 dim)")
print("   • 'clip-vit-base-patch32': Native image-text alignment")

sent_small_bert_L2_128 download started this may take some time.


25/10/22 21:26:50 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.


Approximate size to download 16.1 MB
[ | ]

25/10/22 21:26:51 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.
25/10/22 21:26:51 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.


sent_small_bert_L2_128 download started this may take some time.
Approximate size to download 16.1 MB
Download done! Loading the resource.


[ / ]

2025-10-22 21:26:53.753554: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[OK!]
✅ BERT sentence embeddings configured
   • Model: sent_small_bert_L2_128
   • Dimension: 128
   • Language: English

💡 Alternative models for different use cases:
   • 'all-mpnet-base-v2': Highest quality (384 dim)
   • 'labse': Multilingual support (768 dim)
   • 'clip-vit-base-patch32': Native image-text alignment


## 🔄 Step 6: Build Complete Vision Pipeline

Combine all stages into a cohesive multimodal processing pipeline:

### Pipeline Stages:
1. **Qwen2VLTransformer** → Generate image captions
2. **SentenceDetector** → Split captions into sentences
3. **BertSentenceEmbeddings** → Create vector representations

**Note**: Reader2Image was already executed separately to load images.

In [13]:
# Build the vision processing pipeline
vision_pipe = Pipeline(stages=[
    vlm,        # 1. VLM captioning
    splitter,   # 2. Sentence splitting
    emb         # 3. Embedding generation
])

print("🔧 RAG-Vision pipeline constructed with 3 stages:")
print("   1. Qwen2VLTransformer - Image captioning")
print("   2. SentenceDetector - Caption splitting")
print("   3. BertSentenceEmbeddings - Vector generation")
print("\n✅ Pipeline ready for execution")

🔧 RAG-Vision pipeline constructed with 3 stages:
   1. Qwen2VLTransformer - Image captioning
   2. SentenceDetector - Caption splitting
   3. BertSentenceEmbeddings - Vector generation

✅ Pipeline ready for execution


## ⚙️ Step 7: Execute Vision Pipeline

Run the pipeline to generate captions and embeddings for all images:

### What Happens:
1. **VLM Processing**: Each image is analyzed and captioned
2. **Caption Splitting**: Long descriptions are split into sentences
3. **Embedding Generation**: Each sentence becomes a 128-dimensional vector

**⚠️ Note**: VLM inference can be slow (several seconds per image). For large datasets, consider:
- GPU acceleration
- Batch processing
- Parallel execution

In [14]:
# Execute the vision pipeline
print("🔄 Executing RAG-Vision pipeline...")
print("   ⚠️ VLM processing may take several minutes for the first run")
print("   (downloading models and processing images)\n")

df_vec = vision_pipe.fit(df_images).transform(df_images)

print("✅ Pipeline execution complete!")
print("\n📊 Result DataFrame columns:")
print(f"   • vlm_caption: Generated captions")
print(f"   • sentences: Split caption sentences")
print(f"   • sentence_embeddings: Vector representations")

🔄 Executing RAG-Vision pipeline...
   ⚠️ VLM processing may take several minutes for the first run
   (downloading models and processing images)

✅ Pipeline execution complete!

📊 Result DataFrame columns:
   • vlm_caption: Generated captions
   • sentences: Split caption sentences
   • sentence_embeddings: Vector representations


In [15]:
df_vec.show()

+--------------------+--------------------+---------+--------------------+--------------------+--------------------+
|            fileName|               image|exception|         vlm_caption|           sentences| sentence_embeddings|
+--------------------+--------------------+---------+--------------------+--------------------+--------------------+
|pdf-with-2images.pdf|[{image, pdf-with...|     NULL|[{document, 0, 57...|[{document, 0, 57...|[{sentence_embedd...|
|pdf-with-2images.pdf|[{image, pdf-with...|     NULL|[{document, 0, 50...|[{document, 0, 50...|[{sentence_embedd...|
+--------------------+--------------------+---------+--------------------+--------------------+--------------------+



## 📊 Step 10: Flatten to Chunk-Embedding Pairs

Transform nested structure into flat format for database storage:

### Flattening Process:
1. **Extract Embeddings**: Pull float arrays from annotation objects
2. **Zip Pairs**: Combine sentence text with corresponding embeddings
3. **Explode**: Create one row per (caption_sentence, embedding) pair
4. **Attach Metadata**: Include image properties with each row

### Result Schema:
```
chunk_text: string              # Caption sentence
embedding: array<float>         # 128-dimensional vector
image_uri: string               # Source image path
figure_id: string               # Unique figure identifier
slide_no: string                # Slide number (nullable)
img_w, img_h, img_c: integer    # Image dimensions
ocr_text: string                # OCR text (nullable)
has_image: boolean              # Always true
pipeline: string                # "rag_vision_qwen2vl"
```

In [17]:
print("📊 Flattening data to chunk-embedding pairs...\n")

# Extract raw float arrays from embeddings
df_vec = df_vec.withColumn(
    "emb_vecs",
    F.expr("transform(sentence_embeddings, x -> x.embeddings)")
)

# Zip sentences with their embeddings
df_vec = df_vec.withColumn(
    "pairs",
    F.arrays_zip(F.col("sentences.result"), F.col("emb_vecs"))
)

# Explode to one row per chunk
df_chunks = df_vec \
    .withColumn("pair", F.explode_outer("pairs")) \
    .select(
        F.col("pair.result").alias("chunk_text"),
        F.col("pair.emb_vecs").alias("embedding")
    ) \
    .dropna(subset=["chunk_text", "embedding"])

print("✅ Data flattened to chunk-embedding pairs")
df_chunks.select("chunk_text", "embedding").show(5, truncate=100)

📊 Flattening data to chunk-embedding pairs...

✅ Data flattened to chunk-embedding pairs


[Stage 16:>                                                         (0 + 1) / 1]

+----------------------------------------------------------+----------------------------------------------------------------------------------------------------+
|                                                chunk_text|                                                                                           embedding|
+----------------------------------------------------------+----------------------------------------------------------------------------------------------------+
|A chocolate doughnut with sprinkles on a light background.|[4.3523312E-4, -0.47155252, -0.018804293, -0.8252269, -0.14540172, 1.0212879, 0.39352003, 0.20743...|
|       A blue rocket with a star on it, floating in space.|[-0.089734964, -0.11982616, -0.26973015, -1.2441063, -0.032506846, 0.49904716, -0.89480877, 0.662...|
+----------------------------------------------------------+----------------------------------------------------------------------------------------------------+



## 💾 Step 11: Persist to Storage

Save processed embeddings and metadata to **Parquet format**:

### Storage Benefits:
- **Columnar Format**: Efficient querying by metadata fields
- **Compression**: Reduce storage costs
- **Schema Preservation**: Maintain data types and structure
- **Fast Reads**: Optimized for vector database ingestion

### Next Steps:
1. Load Parquet into vector database (Pinecone, Weaviate, Milvus)
2. Index embeddings for similarity search
3. Build multimodal RAG application
4. Enable visual search capabilities

In [18]:
# Define output path
out_path = "datasets/rag_vision_qwen2vl.parquet"

# Save to Parquet
print(f"💾 Saving results to: {out_path}")
df_chunks.write.mode("overwrite").parquet(out_path)

print(f"\n✅ Successfully saved {df_chunks.count()} chunks to: {out_path}")

💾 Saving results to: datasets/rag_vision_qwen2vl.parquet


[Stage 18:>                                                         (0 + 1) / 1]


✅ Successfully saved 2 chunks to: datasets/rag_vision_qwen2vl.parquet


## 🎓 Key Takeaways and Best Practices

### What We Accomplished:
1. ✅ Built a multimodal RAG pipeline for visual content
2. ✅ Used VLM (Qwen2-VL) to generate contextual image captions
3. ✅ Created searchable embeddings from visual descriptions
4. ✅ Preserved rich metadata for traceability
5. ✅ Enabled dual storage (VLM captions + OCR text)
6. ✅ Prepared data for vector database ingestion

### RAG-Vision vs Text-Only RAG:

| Aspect | Text-Only RAG | RAG-Vision |
|--------|---------------|------------|
| **Content Coverage** | Text only | Text + Images |
| **Chart Understanding** | ❌ Blind | ✅ VLM-powered |
| **Visual Search** | ❌ Not possible | ✅ Enabled |
| **Metadata** | Text-based | Image properties |
| **Cost** | Lower | Higher (VLM) |
| **Complexity** | Simple | Moderate |

### When to Combine Pipelines:

#### Hybrid Strategy: Best Results
- Use **RAG-Base** for text content
- Use **RAG-Vision** for images/charts
- Use **RAG-Boost** for executive summaries
- **Store all in one database** with `has_image` flag

### Production Considerations:

#### 1. VLM Selection
- **Qwen2-VL**: Best for general images and charts
- **GPT-4V**: Higher quality, higher cost
- **LLaVA**: Open-source alternative
- **Gemini Vision**: Google's multimodal model

#### 2. OCR Integration
- **When to add OCR**:
  - ✅ Forms with text fields
  - ✅ Charts with labels and legends
  - ✅ Scanned documents
  - ✅ Screenshots with UI text

#### 3. Metadata Strategy
- **Always include**:
  - `image_uri`: Source traceability
  - `has_image`: Filter flag
  - `figure_id`: Unique identifier
- **Domain-specific**:
  - Medical: `patient_id`, `modality`, `body_part`
  - E-commerce: `product_id`, `category`, `color`
  - Technical: `diagram_type`, `system`, `version`

#### 4. Quality Assurance
- ✅ Review sample captions for accuracy
- ✅ Validate embedding dimensions
- ✅ Check metadata completeness
- ✅ Test retrieval quality with sample queries
- ✅ Monitor VLM hallucinations

#### 5. Performance Optimization
- **GPU Acceleration**: 10-100x faster VLM inference
- **Batch Processing**: Process multiple images simultaneously
- **Caching**: Store captions to avoid reprocessing
- **Async Processing**: Don't block on VLM calls

---

## 🚀 Next Steps: Building Multimodal RAG

### Step 1: Load to Vector Database

```python
# Example: Loading to Pinecone
import pinecone

pinecone.init(api_key="your-api-key")
index = pinecone.Index("multimodal-rag")

# Load embeddings
df = spark.read.parquet("datasets/rag_vision_qwen2vl.parquet")

for row in df.collect():
    index.upsert(
        vectors=[(
            f"{row.figure_id}_{hash(row.chunk_text)}",
            row.embedding,
            {
                "text": row.chunk_text,
                "image_uri": row.image_uri,
                "figure_id": row.figure_id,
                "slide_no": row.slide_no,
                "has_image": row.has_image,
                "ocr_text": row.ocr_text
            }
        )]
    )
```

### Step 2: Implement Visual Search

```python
def visual_search(query: str, top_k: int = 5, filter_images_only: bool = False):
    # Embed query
    query_embedding = model.encode(query)
    
    # Build filter
    filter_dict = {"has_image": True} if filter_images_only else None
    
    # Search
    results = index.query(
        vector=query_embedding.tolist(),
        top_k=top_k,
        filter=filter_dict,
        include_metadata=True
    )
    
    return results
```

### Step 3: Build Multimodal RAG

```python
def multimodal_rag(user_question: str):
    # 1. Retrieve relevant content (text + images)
    results = visual_search(user_question, top_k=5)
    
    # 2. Separate text and image results
    text_context = []
    image_references = []
    
    for match in results['matches']:
        meta = match['metadata']
        if meta.get('has_image'):
            image_references.append({
                'caption': meta['text'],
                'image_uri': meta['image_uri'],
                'figure_id': meta['figure_id']
            })
        text_context.append(meta['text'])
    
    # 3. Build rich context
    context = "\n\n".join([
        f"[{i+1}] {text}"
        for i, text in enumerate(text_context)
    ])
    
    # 4. Generate answer with image citations
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content":
             "You are a helpful assistant with access to images and charts. "
             "When referencing visual content, cite the figure_id."},
            {"role": "user", "content":
             f"Context:\n{context}\n\nQuestion: {user_question}"}
        ]
    )
    
    return {
        'answer': response.choices[0].message.content,
        'image_references': image_references
    }
```

### Step 4: Display Results with Images

```python
result = multimodal_rag("What were the Q4 sales figures?")

print("Answer:", result['answer'])
print("\nReferenced Images:")
for img_ref in result['image_references']:
    print(f"  • {img_ref['figure_id']}: {img_ref['caption']}")
    # Display image: img_ref['image_uri']
```

---

## 📚 Additional Resources

### Documentation:
- [Spark NLP Documentation](https://nlp.johnsnowlabs.com/)
- [Qwen2-VL Model Card](https://huggingface.co/Qwen/Qwen2-VL)
- [Reader2Image Guide](https://nlp.johnsnowlabs.com/docs/en/readers)

### Vision-Language Models:
- [Qwen2-VL](https://github.com/QwenLM/Qwen2-VL): Open-source VLM
- [GPT-4V](https://platform.openai.com/docs/guides/vision): OpenAI's vision model
- [LLaVA](https://llava-vl.github.io/): Large Language and Vision Assistant
- [CLIP](https://github.com/openai/CLIP): Contrastive image-text learning

### Multimodal RAG:
- [LangChain Multimodal](https://python.langchain.com/docs/use_cases/multimodal)
- [LlamaIndex Vision](https://docs.llamaindex.ai/en/stable/examples/multi_modal/)
- [Pinecone Multimodal Search](https://www.pinecone.io/learn/multimodal-search/)

### OCR Tools:
- [Spark OCR](https://nlp.johnsnowlabs.com/docs/en/ocr)
- [Tesseract](https://github.com/tesseract-ocr/tesseract)
- [AWS Textract](https://aws.amazon.com/textract/)
- [Google Vision API](https://cloud.google.com/vision)

---

## 🎉 Congratulations!

You've successfully built a **multimodal RAG-Vision pipeline** that unlocks the semantic content of images. This enables:

✅ **Visual Search**: Find charts and diagrams by description  
✅ **Complete Coverage**: Index both text and visual content  
✅ **Rich Citations**: Reference figures with context  
✅ **Multimodal Q&A**: Answer questions about visual data  

**Next**: Combine with RAG-Base and RAG-Boost for comprehensive enterprise RAG!

**Happy Building! 🚀**